In [27]:
import tensorflow as tf
import numpy as np
import os
from numpy import genfromtxt
from keras import backend as K
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate,Permute,Reshape,Cropping2D,Maximum,Cropping1D
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
import fr_utils
from keras.layers.core import Lambda, Flatten, Dense, Dropout
#from theano.tensor import _shared
from keras.utils import to_categorical
import pickle

In [17]:
m=open('x_train.pkl','rb+')
x_train=pickle.load(m)

In [16]:
y_train=open('y_train.pkl','rb+')
y_train=pickle.load(y_train)

In [22]:
labels=pickle.load(open('labels.pkl','rb'))

In [ ]:
x_test=pickle.load(open('x_test.pkl','rb'))
y_test=pickle.load(open('y_test.pkl','rb'))

In [29]:
def conv2d_bn(x, filters, kernel_size, strides=1, padding='same', activation='relu', use_bias=False, name=None):
    x = Conv2D(filters, kernel_size, strides=strides, padding=padding, use_bias=use_bias, kernel_initializer='he_normal', name=name)(x)
    if not use_bias:
        bn_axis = 1 if K.image_data_format() == 'channels_first' else 3
        bn_name = None if name is None else name + '_bn'
        x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    return x

def mfm(x):
    shape = K.int_shape(x)
    x = Permute(dims=(3, 2, 1))(x) # swap 1 <-> 3 axis
    x1 = Cropping2D(cropping=((0, shape[3] // 2), 0))(x)
    x2 = Cropping2D(cropping=((shape[3] // 2, 0), 0))(x)
    x = Maximum()([x1, x2])
    x = Permute(dims=(3, 2, 1))(x) # swap 1 <-> 3 axis
    x = Reshape([shape[1], shape[2], shape[3] // 2])(x)
    return x

def common_conv2d(net, filters, filters2):
    res = net

    net = conv2d_bn(net, filters=filters, kernel_size=1, strides=1, padding='same')
    net = mfm(net)
    net = conv2d_bn(net, filters=filters2, kernel_size=3, strides=1, padding='same')
    net = mfm(net)

    return net

def lcnn9(inputs):
    # Conv1
    net = conv2d_bn(inputs, filters=96, kernel_size=5, strides=1, padding='same')
    net = mfm(net)
    net = MaxPooling2D(pool_size=2, strides=2, padding='same')(net)
    net = Dropout(0.3)(net)

    # Block1
    net = common_conv2d(net,filters=96, filters2=192)
    net = MaxPooling2D(pool_size=2, strides=2, padding='same')(net)
    net = Dropout(0.3)(net)
    # Block2
    net = common_conv2d(net,filters=192, filters2=384)
    net = MaxPooling2D(pool_size=2, strides=2, padding='same')(net)
    net = Dropout(0.3)(net)
    # Block3
    net = common_conv2d(net,filters=384, filters2=256)

    # Block4
    net = common_conv2d(net,filters=256, filters2=256)
    net = MaxPooling2D(pool_size=2, strides=2, padding='same')(net)

    net = Flatten()(net)

    return net


In [30]:
input_image = Input(shape=(64, 64, 3))

lcnn_output = lcnn9(inputs=input_image)

fc1 = Dense(512, activation=None)(lcnn_output)
fc1 = Reshape((512, 1))(fc1)
fc1_1 = Cropping1D(cropping=(0, 256))(fc1)
fc1_2 = Cropping1D(cropping=(256, 0))(fc1)
fc1 = Maximum()([fc1_1, fc1_2])
fc1 = Flatten()(fc1)

out = Dense(610, activation='softmax')(fc1)
model = Model(inputs=[input_image], outputs=out)

In [32]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x_train[0],to_categorical(y_train[0],610),nb_epoch=10,batch_size=64)

D:\deeplearning\anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/10
6123/6123 [==============================] - 42s 7ms/step - loss: 1.2353 - acc: 0.6619
Epoch 2/10
6123/6123 [==============================] - 40s 7ms/step - loss: 0.6973 - acc: 0.8016
Epoch 3/10
6123/6123 [==============================] - 41s 7ms/step - loss: 0.4930 - acc: 0.8600
Epoch 4/10
6123/6123 [==============================] - 41s 7ms/step - loss: 0.3376 - acc: 0.9058
Epoch 5/10
6123/6123 [==============================] - 41s 7ms/step - loss: 0.2450 - acc: 0.9286
Epoch 6/10
6123/6123 [==============================] - 41s 7ms/step - loss: 0.1624 - acc: 0.9580
Epoch 7/10
6123/6123 [==============================] - 41s 7ms/step - loss: 0.1616 - acc: 0.9543
Epoch 8/10
6123/6123 [==============================] - 41s 7ms/step - loss: 0.1318 - acc: 0.9639
Epoch 9/10
6123/6123 [==============================] - 41s 7ms/step - loss: 0.1508 - acc: 0.9543
Epoch 10/10
6123/6123 [==============================] - 41s 7ms/step - loss: 0.1969 - acc: 0.9427


In [40]:
model.evaluate(x_test,to_categorical(y_test,610),batch_size=64)

610/610 [==============================] - 2s 3ms/step


[9.8014344574975194, 0.080327868974599675]

In [39]:
import cv2
cv2.imwrite('t.jpg',x_test[0])


True